In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 10
verbosity = 2
input_shape = (width, height, channel)

In [3]:
pretrained_model = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')


pretrained_model.trainable=False

model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.


In [7]:
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    verbose=verbosity,
                   )

Epoch 1/10
40/40 - 22s - loss: 0.1316 - auc: 0.9984 - accuracy: 0.9748
Epoch 2/10
40/40 - 22s - loss: 0.1214 - auc: 0.9990 - accuracy: 0.9807
Epoch 3/10
40/40 - 22s - loss: 0.1111 - auc: 0.9995 - accuracy: 0.9870
Epoch 4/10
40/40 - 22s - loss: 0.1056 - auc: 0.9995 - accuracy: 0.9811
Epoch 5/10
40/40 - 22s - loss: 0.0987 - auc: 0.9996 - accuracy: 0.9886
Epoch 6/10
40/40 - 22s - loss: 0.0926 - auc: 0.9998 - accuracy: 0.9909
Epoch 7/10
40/40 - 22s - loss: 0.0857 - auc: 0.9999 - accuracy: 0.9902
Epoch 8/10
40/40 - 22s - loss: 0.0853 - auc: 0.9997 - accuracy: 0.9886
Epoch 9/10
40/40 - 23s - loss: 0.0744 - auc: 0.9999 - accuracy: 0.9941
Epoch 10/10
40/40 - 22s - loss: 0.0724 - auc: 0.9999 - accuracy: 0.9957


In [8]:
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

model.evaluate(valid_generator)

Found 650 images belonging to 2 classes.
11/11 [==============================] - 5s 493ms/step - loss: 0.0624 - auc: 1.0000 - accuracy: 0.9985


[0.062382400035858154, 1.0, 0.9984615445137024]

In [9]:
prediction = model.predict(valid_generator)

In [10]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

319
331
